In [ ]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 401 kB 14.5 MB/s 
     |████████████████████████████████| 19.7 MB 390 kB/s 
     |████████████████████████████████| 788 kB 65.3 MB/s 
     |████████████████████████████████| 163 kB 68.2 MB/s 
     |████████████████████████████████| 1.2 MB 57.7 MB/s 
     |████████████████████████████████| 981 kB 64.3 MB/s 
     |████████████████████████████████| 46 kB 4.5 MB/s 
     |████████████████████████████████| 1.6 MB 54.5 MB/s 
     |████████████████████████████████| 5.5 MB 37.8 MB/s 
     |████████████████████████████████| 53 kB 1.5 MB/s 
     |████████████████████████████████| 200 kB 73.5 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 7.6 MB 51.2 MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-py3-none-any.whl size=116702 sha256=ded23b0438c33fde4af4f1898d38359491b096a073317ad081c0402fa9c08fa0
  Stor

In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
columns = {0: 'text', 1: 'ner'}
corpus: Corpus = ColumnCorpus('/content/data',columns,
                              train_file='/content/TRAIN.txt',
                              dev_file='/content/DEV.txt',
                              test_file='/content/DEV.txt')

2022-11-14 03:48:56,429 Reading data from /content/data
2022-11-14 03:48:56,433 Train: /content/IDRIS_train_new_combined.txt
2022-11-14 03:48:56,439 Dev: /content/IDRIS_test_new_combined.txt
2022-11-14 03:48:56,444 Test: /content/IDRIS_test_new_combined.txt


In [ ]:
import pandas as pd
data = [[len(corpus.train),  len(corpus.dev)]]
# Prints out the dataset sizes of train test and development in a table.
pd.DataFrame(data, columns=["Train", "Development"])

,Train,Development
0,14392,2056


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.embeddings import WordEmbeddings, StackedEmbeddings, FlairEmbeddings
import torch

tag_type = 'ner'

tag_dictionary = corpus.make_label_dictionary(label_type=tag_type)

from flair.embeddings import TransformerWordEmbeddings, WordEmbeddings

embedding_types = [ 
   TransformerWordEmbeddings(
    model='tner/deberta-v3-large-ontonotes5',
    layers="-1",
    subtoken_pooling="first",
    fine_tune=True,
    use_context=True,
),
    glove_embedding = WordEmbeddings('glove'),
    TransformerWordEmbeddings('dslim/bert-base-NER'),
]
embeddings = StackedEmbeddings(embeddings=embedding_types)

from flair.models import SequenceTagger
tagger = SequenceTagger(
    hidden_size=768,
    embeddings=embeddings,
    tag_dictionary=tag_dictionary,
    tag_type='ner',
    use_crf=False,
    use_rnn=False,
    reproject_embeddings=True,
)


from flair.trainers import ModelTrainer
from torch.optim import AdamW
trainer = ModelTrainer(tagger, corpus)

from torch.optim.lr_scheduler import OneCycleLR

trainer.train('/content/drive/resources/taggers/ner-english-ontonotes-large',
              learning_rate=5.0e-6,
              mini_batch_size=4,
              mini_batch_chunk_size=1,
              max_epochs=3,
              optimizer=AdamW,
              scheduler=OneCycleLR,
              embeddings_storage_mode='none',
              weight_decay=0.,
              )


2022-11-14 03:49:10,570 Computing label dictionary. Progress:


14392it [00:00, 49705.81it/s]

2022-11-14 03:49:10,917 Dictionary created for label 'ner' with 2 values: LOC (seen 15363 times)


Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

...

2022-11-14 07:07:25,553 Evaluating as a multi-label problem: False
2022-11-14 07:07:25,577 DEV : loss 0.05208056420087814 - f1-score (micro avg)  0.9036
2022-11-14 07:07:25,636 BAD EPOCHS (no improvement): 4


2022-11-14 07:07:33,162 ----------------------------------------------------------------------------------------------------
2022-11-14 07:07:33,359 loading file resources/taggers/ner-english-ontonotes-large/best-model.pt
2022-11-14 07:07:48,471 SequenceTagger predicts: Dictionary with 5 tags: O, S-LOC, B-LOC, E-LOC, I-LOC


100%|██████████| 514/514 [02:37<00:00,  3.26it/s]

2022-11-14 07:10:27,027 Evaluating as a multi-label problem: False
2022-11-14 07:10:27,053 0.892	0.9184	0.905	0.8265
2022-11-14 07:10:27,055 
Results:
- F-score (micro) 0.905
- F-score (macro) 0.905
- Accuracy 0.8265

By class:
              precision    recall  f1-score   support

         LOC     0.8920    0.9184    0.9050      2194

   micro avg     0.8920    0.9184    0.9050      2194
   macro avg     0.8920    0.9184    0.9050      2194
weighted avg     0.8920    0.9184    0.9050      2194

2022-11-14 07:10:27,058 ----------------------------------------------------------------------------------------------------


{'test_score': 0.9050078598697507,
 'dev_score_history': [0.9038770360174352,
  0.9050078598697507,
  0.903584947673124],
 'train_loss_history': [0.133333344416012,
  0.10519108530888953,
  0.09468267186836915],
 'dev_loss_history': [0.048650018870830536,
  0.04549569636583328,
  0.05208056420087814]}

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

input_sentence = "In the wake of #HurricaneHarvey's devastation we're taking action. Join us in donating towards relief efforts: at the walmart in College Station tomorrow."
sentence: Sentence = Sentence(input_sentence)
tagger.predict(sentence)
print(sentence.to_tagged_string())

Sentence: "In the wake of # HurricaneHarvey 's devastation we 're taking action . Join us in donating towards relief efforts : at the walmart in College Station tomorrow ." → ["College Station"/LOC]


In [ ]:
tagger.save('drive/MyDrive/model.pt')

In [ ]:
import os
os.listdir()

['.config',
 'drive',
 'resources',
 'IDRIS_test_new_combined.txt',
 'IDRIS_train_new_combined.txt',
 'sample_data']